In [1]:
import pandas as pd 
from sklearn.neighbors import NearestNeighbors
from sqlalchemy import create_engine
import pickle
from fastapi import FastAPI
import threading
import time

In [2]:
app = FastAPI()

In [3]:
knn = None 
user_item_matrix = None
restaurantsData = None

In [4]:
connection_string = "mssql+pyodbc://@DESKTOP-OAFR7NA/DealsHubDB?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server"
engine = create_engine(connection_string)

In [ ]:
def fetch_reviews():
    try:
        ##query = "SELECT ReviewId, Rating, UserId, BusinessId, CreatedAt FROM Reviews"
        ##data = pd.read_sql(query, engine)
        print("Reviews fetched successfully.")
        return data
    except Exception as e:
        print(f"Error in fetching reviews: {e}")
        return None

In [29]:
def fetch_Businesses():
    try:
        query = "SELECT BusinessId, Area FROM Businesses"
        data = pd.read_sql(query, engine)
        print("Businesses fetched successfully.")
        return data
    except Exception as e:
        print(f"Error in fetching businesses: {e}")
        return None

In [31]:
def retrain_user_model():
    global knn, user_item_matrix, businessesData

    data = fetch_reviews()

    if data is None or data.empty:
        print("No data available for training.")
        return None

    data["createdAt"] = pd.to_datetime(data["createdAt"])
    data = data.sort_values("createdAt").drop_duplicates(subset=["UserId", "BusinessId"], keep="last") 

    user_item_matrix = data.pivot(index='UserId', columns='BusinessId', values='Rating').fillna(0)

    knn = NearestNeighbors(metric="cosine", algorithm="brute", n_neighbors=5)
    knn.fit(user_item_matrix)

    pickle.dump(knn, open(MODEL_FILE, "wb"))
    print("Model retrained and saved successfully.")
    return knn

In [35]:
def recommendRestaurantsToUserInArea(userId: int, area: str, numOfRecommends: int = 6):
    if userId not in user_item_matrix.index:
        return "User Not Found"

    user_idx = user_item_matrix.index.get_loc(userId)
    distances, indices = knn.kneighbors([user_item_matrix.iloc[user_idx]], n_neighbors=6)

    similar_users = user_item_matrix.iloc[indices.flatten()[1:]]
    
    recommended_restaurants = (similar_users.mean().sort_values(ascending=False)).index

    filtered_restaurants = restaurantsData[restaurantsData['Area'] == area]['Restaurant ID']

    final_recommendations = [r for r in recommended_restaurants if r in filtered_restaurants.values]

    return final_recommendations[:numOfRecommends]

In [47]:
def recommendRestaurantsToUser(userId: int, numOfRecommends: int = 6):
    if userId not in user_item_matrix.index:
        return "User Not Found"

    user_idx = user_item_matrix.index.get_loc(userId)
    distances, indecies = knn.kneighbors([user_item_matrix.iloc[user_idx]], n_neighbors=6)

    similar_users = user_item_matrix.iloc[indecies.flatten()[1:]]
    
    recommend_restuarants = (similar_users.mean().sort_values(ascending=False)).index[:numOfRecommends]

    return recommend_restuarants.tolist()

In [43]:
def retrain_model_periodically(interval=3600):
    while True:
        print("Retraining the model in the background...")
        retrain_user_model()
        time.sleep(interval)

In [45]:
threading.Thread(target=retrain_model_periodically, daemon=True).start()

Retraining the model in the background...


In [51]:
@app.get("/userRecommendWithArea")
async def userRecommendWithArea(userId: int, area: str, numOfRecommends: int = 6):
    recommendations = recommendRestaurantsToUserInArea(userId, area, numOfRecommends)
    return recommendations

In [53]:
@app.get("/userRecommend")
async def userRecommend(userId: int, numOfRecommends: int = 6):
    recommendations = recommendRestaurantsToUser(userId, numOfRecommends)
    return recommendations

In [47]:
data["createdAt"] = pd.to_datetime(data["createdAt"])

In [49]:
data = data.sort_values("createdAt").drop_duplicates(subset=["User ID", "Restaurant ID"], keep="last")
data

,User ID,Restaurant ID,Rating,Review,createdAt
152678,2144,835,3,Great food and experience!,2020-03-04
82906,5086,857,1,Great food and experience!,2020-03-04
494315,2144,393,2,Great food and experience!,2020-03-04
145953,3991,762,2,Great food and experience!,2020-03-04
432289,7751,688,4,Great food and experience!,2020-03-05
...,...,...,...,...,...
370235,6797,767,1,Great food and experience!,2025-03-17
264666,1810,482,2,Great food and experience!,2025-03-17
477959,6389,984,4,"Juicy and well-cooked meat, really good.",2025-03-17
65000,112,997,2,Great food and experience!,2025-03-17


In [4]:
restaurantsData = pd.read_csv("GeneratedData/egyptian_restaurants_refined.csv")
restaurantsData

,Restaurant ID,Restaurant Name,Category,City,Area,Longitude,Latitude
0,1,30 North,Cafe,Alexandria,Gleem,29.901193,31.186937
1,2,Indira,Asian Cuisine,Alexandria,Montaza,29.920624,31.206004
2,3,Seoudi,Casual Dining,Cairo,Nasr City,31.245766,30.041615
3,4,Paul,Bakery,Alexandria,Sidi Gaber,29.906400,31.189993
4,5,Wel3a,Grill,Sharm El Sheikh,Sharks Bay,34.325296,27.906238
...,...,...,...,...,...,...,...
995,996,Koshary Hend,Koshary,Giza,Dokki,31.217653,30.022743
996,997,Abou Haidar,Sandwiches,Hurghada,El Gouna,33.814798,27.261863
997,998,Maison de Curry,Indian,Cairo,Heliopolis,31.240446,30.026338
998,999,Arabiata,Egyptian Cuisine,Cairo,Downtown,31.228161,30.045418


In [5]:
user_item_matrix = data.pivot(index='User ID', columns='Restaurant ID', values='Rating').fillna(0)

In [6]:
knn = NearestNeighbors(metric="cosine", algorithm="brute", n_neighbors=5)
knn.fit(user_item_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [7]:
def recommendRestaurantsToUser(userId, numOfRecommends):
    if userId not in user_item_matrix.index:
        return "User Not Found"

    user_idx = user_item_matrix.index.get_loc(userId)
    distances, indecies = knn.kneighbors([user_item_matrix.iloc[user_idx]], n_neighbors=6)

    similar_users = user_item_matrix.iloc[indecies.flatten()[1:]]
    
    recommend_restuarants = (similar_users.mean().sort_values(ascending=False)).index[:numOfRecommends]

    return recommend_restuarants.tolist()

In [39]:
recommendations = recommendRestaurantsToUser(555,5)
recommendations

[937, 766, 500, 652, 812]

In [41]:
restaurantsData[restaurantsData["Restaurant ID"].isin(recommendations)]

,Restaurant ID,Restaurant Name,Category,City,Area,Longitude,Latitude
499,500,Tamara,Lebanese,Alexandria,Sidi Gaber,29.907117,31.189543
651,652,Buffalo Burger,Burgers,Hurghada,Sheraton Road,33.809546,27.240095
765,766,Butcher's Burger,Burgers,Giza,Haram,31.206092,30.008051
811,812,Koshary Hend,Koshary,Giza,Haram,31.215088,30.012383
936,937,Bazooka,Burgers,Cairo,Zamalek,31.241822,30.027337


In [21]:
def recommendRestaurantsToUserInArea(userId, numOfRecommends, area):
    if userId not in user_item_matrix.index:
        return "User Not Found"

    user_idx = user_item_matrix.index.get_loc(userId)
    distances, indices = knn.kneighbors([user_item_matrix.iloc[user_idx]], n_neighbors=6)

    similar_users = user_item_matrix.iloc[indices.flatten()[1:]]
    
    recommended_restaurants = (similar_users.mean().sort_values(ascending=False)).index

    filtered_restaurants = restaurantsData[restaurantsData['Area'] == area]['Restaurant ID']

    final_recommendations = [r for r in recommended_restaurants if r in filtered_restaurants.values]

    return final_recommendations[:6]  

In [33]:
recommendations = recommendRestaurantsToUserInArea(555,5,"Zamalek")
recommendations

[937, 829, 34, 236, 895, 691]

In [35]:
restaurantsData[restaurantsData["Restaurant ID"].isin(recommendations)]

,Restaurant ID,Restaurant Name,Category,City,Area,Longitude,Latitude
33,34,Butcher's Burger,Burgers,Cairo,Zamalek,31.223389,30.047763
235,236,Hossam El Refaay,Grill,Cairo,Zamalek,31.251662,30.038643
690,691,Chicken Fil-A,Fried Chicken,Cairo,Zamalek,31.216639,30.036275
828,829,Fuddruckers,Burgers,Cairo,Zamalek,31.220837,30.044145
894,895,Gad,Egyptian Cuisine,Cairo,Zamalek,31.241175,30.039210
936,937,Bazooka,Burgers,Cairo,Zamalek,31.241822,30.027337


In [31]:
ratedRestaurants = data[data["User ID"]==555][["Restaurant ID","Rating"]]
ratedRestaurants = ratedRestaurants.merge(restaurantsData, on="Restaurant ID")
ratedRestaurants.sort_values(by="Rating", ascending=False)

,Restaurant ID,Rating,Restaurant Name,Category,City,Area,Longitude,Latitude
28,45,5,Kazouza,Egyptian Street Food,Giza,Haram,31.221621,30.024833
11,937,5,Bazooka,Burgers,Cairo,Zamalek,31.241822,30.027337
47,942,5,Makani,Grill,Sharm El Sheikh,Nabq,34.329407,27.934191
42,406,5,Starbucks,Cafe,Giza,Mohandessin,31.215055,30.031872
39,238,5,Tabla Luna,Latin American,Giza,Dokki,31.224261,30.032845
31,516,5,Koshary Hend,Koshary,Sharm El Sheikh,Naama Bay,34.344990,27.929137
29,851,5,The Lemon Tree,Mediterranean,Hurghada,Dahar,33.796181,27.243677
23,855,5,Wel3a,Grill,Giza,6th of October,31.212438,30.032115
19,109,5,Seoudi,Casual Dining,Hurghada,Dahar,33.815551,27.254341
12,812,5,Koshary Hend,Koshary,Giza,Haram,31.215088,30.012383
